In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_two_column_with_rank_11_withoutDT.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  127.425882     7.040131
1  233.628927    12.907725
2  134.062123     7.406775
3  245.587090    13.568399
4  364.485990    20.137424


MAP THE LOAD OF MEANHLY AVERGAE WITH THE FEEDER

In [ ]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


COMMULATIVE DIFFERENCE FOR MONTHLY AVERAGE LOAD OF SCADA


In [3]:
import pandas as pd

# File paths
feeder_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/fedder_11KV_with_DT_LOAD_CAL.csv"
monthly_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv"
output_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv"


# Load data
feeder_df = pd.read_csv(feeder_path)
monthly_df = pd.read_csv(monthly_path)

# Merge on FEEDER_ID <-> SWNO
merged = feeder_df.merge(monthly_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

month_cols = [f'Month_{i:02d}' for i in range(1, 13)]

result_rows = []

for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
    group = group.copy()
    group['__orig_idx'] = group.index
    ranks_in_order = group['RANK'].drop_duplicates().tolist()

    dt_load_sums = {
        rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
        for rank in ranks_in_order
    }

    prev_cum = {m: None for m in month_cols}

    for rank in ranks_in_order:
        this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
        first = True
        for idx, row in this_rank_rows.iterrows():
            base = {col: row[col] for col in feeder_df.columns}
            for m in month_cols:
                month_val = row.get(m, None)
                if first:
                    # Only for the first row of this rank
                    if rank == ranks_in_order[0]:
                        cum_val = month_val - dt_load_sums[rank] if pd.notna(month_val) else None
                    else:
                        prev = prev_cum[m]
                        cum_val = prev - dt_load_sums[rank] if prev is not None else None
                    prev_cum[m] = cum_val
                else:
                    cum_val = None  # For subsequent rows at same rank, leave blank
                base[f"{m}_cumulative_diff"] = cum_val
            result_rows.append(base)
            first = False

result_df = pd.DataFrame(result_rows)
output_cols = list(feeder_df.columns) + [f"{m}_cumulative_diff" for m in month_cols]
result_df.to_csv(output_path, index=False, columns=output_cols)
print(f"Saved to {output_path}")


Saved to /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/DATA_GENERATION/final_feeder_11kv_monthly_cumulative_diff.csv


AFTER THE RANKING AND USE ONLY MAIN CHAIN DONT USING SUBCHAIN

In [1]:
import pandas as pd
import numpy as np

# 1. Read your CSV
df = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")

# 2. Compute the constant denominator: 11 * 0.95 * √3
den = 11 * 0.95 * np.sqrt(3)

# 3. Create the new column
df["DT_LOAD_CAL"] = df["DT_LOAD"] / den

# 4. (Optional) inspect first few rows
print(df[["DT_LOAD", "DT_LOAD_CAL"]].head())

# 5. Save to a new CSV (or overwrite)
df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv", index=False)


      DT_LOAD  DT_LOAD_CAL
0  616.320000    34.050959
1  674.080488    37.242158
2  473.442574    26.157148
3  148.855367     8.224085
4  400.698507    22.138124


In [3]:
import pandas as pd

# 1) Load your files
sw = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/CSV FILE/Monthly_average_from_scada/monthly_SWNO_matrix_numpy.csv")       # contains SWNO, Month_01…Month_12
feeder = pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv")      # contains feeder_id, DT_LOAD_CAL

# 2) Build a lookup from feeder_id → DT_LOAD_CAL
dt_map = feeder.set_index("FEEDER_ID")["DT_LOAD_CAL"].to_dict()

# 3) Prepare for the walk:
#    We'll keep track of the "current" DT_LOAD_CAL and the last calculated values for each month.
current_dt = None
prev_vals = {f"Month_{i:02d}": None for i in range(1,13)}

# 4) Iterate rows in order, computing chain‐subtract columns:
out_rows = []
for _, row in sw.iterrows():
    swno = row["SWNO"]
    # If this SWNO is a feeder anchor, reset current_dt and prev_vals
    if swno in dt_map:
        current_dt = dt_map[swno]
        # initialize prev_vals for this anchor: first subtraction
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = row[m] - current_dt
    else:
        # continue chain: subtract the same current_dt from last prev_vals
        for i in range(1,13):
            m = f"Month_{i:02d}"
            prev_vals[m] = prev_vals[m] - current_dt

    # build output row
    out = row.to_dict()
    # add the new _CAL columns
    for i in range(1,13):
        m = f"Month_{i:02d}"
        out[f"{m}_CAL"] = prev_vals[m]
    out_rows.append(out)

# 5) Assemble and write out
out_df = pd.DataFrame(out_rows)
out_df.to_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/feeder_11Kv_sw_with_chain_calculations.csv", index=False)


In [1]:
import pandas as pd

# File paths (change as needed)
feeder_path = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_DT_LOAD_CAL.csv"
daily_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2025.csv"
daily_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/daily_SWNO_matrix_11KV_YEAR2024.csv"
output_path_1 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv"
output_path_2 = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv"


In [2]:
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()


In [3]:
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    output_cols = list(feeder_df.columns) + [f"{d}_cumulative_diff" for d in day_cols]
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# Process each year
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


In [4]:
import pandas as pd

# --- Identify only main-chain feeders (no sub-branches) ---
feeder_df = pd.read_csv(feeder_path)

def is_main_chain_only(ranks):
    return all('.' not in str(r) for r in ranks)

main_chain_flags = (
    feeder_df.groupby('FEEDER_ID')['RANK']
    .apply(is_main_chain_only)
    .rename('IS_MAIN_CHAIN_ONLY')
    .reset_index()
)
main_chain_ids = main_chain_flags[main_chain_flags['IS_MAIN_CHAIN_ONLY']]['FEEDER_ID'].tolist()
main_chain_feeder_df = feeder_df[feeder_df['FEEDER_ID'].isin(main_chain_ids)].copy()

# --- Processing function ---
def process_year(main_chain_feeder_df, daily_path, output_path):
    daily_df = pd.read_csv(daily_path)
    day_cols = [col for col in daily_df.columns if col.startswith("Day_")]

    merged = main_chain_feeder_df.merge(daily_df, left_on='FEEDER_ID', right_on='SWNO', how='left')

    result_rows = []

    for feeder_id, group in merged.groupby('FEEDER_ID', sort=False):
        group = group.copy()
        group['__orig_idx'] = group.index
        ranks_in_order = group['RANK'].drop_duplicates().tolist()

        dt_load_sums = {
            rank: group[group['RANK'] == rank]['DT_LOAD_CAL'].fillna(0).sum()
            for rank in ranks_in_order
        }

        prev_cum = {d: None for d in day_cols}

        for rank in ranks_in_order:
            this_rank_rows = group[group['RANK'] == rank].sort_values('__orig_idx')
            first = True
            for idx, row in this_rank_rows.iterrows():
                base = {col: row[col] for col in main_chain_feeder_df.columns}
                for d in day_cols:
                    day_val = row.get(d, None)
                    if first:
                        if rank == ranks_in_order[0]:
                            cum_val = day_val - dt_load_sums[rank] if pd.notna(day_val) else None
                        else:
                            prev = prev_cum[d]
                            cum_val = prev - dt_load_sums[rank] if prev is not None else None
                        prev_cum[d] = cum_val
                    else:
                        cum_val = None
                    base[f"{d}_cumulative_diff"] = cum_val
                result_rows.append(base)
                first = False

    result_df = pd.DataFrame(result_rows)
    # --- Add FROM_SWITCH and TO_SWITCH columns ---
    if "FROM_TO" in result_df.columns:
        from_to_split = result_df["FROM_TO"].str.split("-", n=1, expand=True)
        result_df["FROM_SWITCH"] = from_to_split[0]
        result_df["TO_SWITCH"] = from_to_split[1]
        cols = list(result_df.columns)
        i = cols.index("FROM_TO") + 1
        for col in ["FROM_SWITCH", "TO_SWITCH"]:
            cols.insert(i, cols.pop(cols.index(col)))
            i += 1
        result_df = result_df[cols]
    # --- Save ---
    output_cols = list(result_df.columns)  # Save all columns, now includes FROM_SWITCH and TO_SWITCH
    result_df.to_csv(output_path, index=False, columns=output_cols)
    print(f"Saved to {output_path}")

# --- Process each year ---
process_year(main_chain_feeder_df, daily_path_1, output_path_1)
process_year(main_chain_feeder_df, daily_path_2, output_path_2)


Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2025.csv
Saved to /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/final_feeder_11kv_daily_cumulative_diff_2024.csv


NEW LOGIC TO CALCULATE THE LOAD

In [8]:
# ── Imports ────────────────────────────────────────────────────────────────
from __future__ import annotations
from pathlib import Path
import re
import numpy as np
import pandas as pd

# ── Paths ───────────────────────────────────────────────────────────────────
BASE      = "/media/sagark24/New Volume/MERGE CDIS"
IN_FEDDER = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")
IN_MATRIX = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/monthly_SWNO_matrix_11KV.csv")
OUT_FILE  = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")

# ── Dynamically read month columns from the matrix header ───────────────────
#     (any column exactly matching "Month_XX")
hdr = pd.read_csv(IN_MATRIX, nrows=0).columns
MONTH_KEYS = sorted(
    [c for c in hdr if re.fullmatch(r"Month_\d{2}", c)],
    key=lambda x: int(x.split("_")[1])
)
if not MONTH_KEYS:
    raise RuntimeError("No Month_XX columns found in the matrix!")

# Build downstream column names that keep the original tag
FEED_COLS = [f"feeder_{m}"    for m in MONTH_KEYS]
NODE_COLS = [f"node_load_{m}" for m in MONTH_KEYS]
CUM_COLS  = [f"sec_cum_{m}"   for m in MONTH_KEYS]

# ── Helpers ─────────────────────────────────────────────────────────────────
def make_feed_long(mat: pd.DataFrame) -> pd.DataFrame:
    """
    Return FEEDER_ID | MONTH | FEEDER_LOAD_AVG (long form),
    where MONTH is the *original* column label (e.g. 'Month_01').
    Fills missing (feeder,month) pairs with 0.
    """
    need = {"SWNO", *MONTH_KEYS}
    miss = [c for c in need if c not in mat.columns]
    if miss:
        raise RuntimeError(f"monthly matrix missing columns: {miss}")

    m = mat.copy()
    m["FEEDER_ID"] = m["SWNO"].astype(str).str.strip()
    for c in MONTH_KEYS:
        m[c] = pd.to_numeric(m[c], errors="coerce").fillna(0.0)

    long = m.melt(id_vars=["FEEDER_ID"], value_vars=MONTH_KEYS,
                  var_name="MONTH", value_name="FEEDER_LOAD_AVG")

    complete = (pd.MultiIndex.from_product(
                    [m["FEEDER_ID"].unique(), MONTH_KEYS],
                    names=["FEEDER_ID", "MONTH"]
                ).to_frame(index=False))
    return (complete.merge(long, on=["FEEDER_ID", "MONTH"], how="left")
                    .fillna({"FEEDER_LOAD_AVG": 0.0}))

def build_graph(df_f: pd.DataFrame) -> dict[str, list[str]]:
    """Adjacency: SOURCE_LOCATION -> [DESTINATION_LOCATION…]."""
    adj: dict[str, list[str]] = {}
    for s, d in zip(df_f["SOURCE_LOCATION"].astype(str),
                    df_f["DESTINATION_LOCATION"].astype(str)):
        adj.setdefault(s, [])
        if d not in adj[s]:
            adj[s].append(d)
        adj.setdefault(d, [])          # ensure leaf key
    return adj

def dfs_subtree_cum(adj, node_vec):
    """Bottom-up DFS returning {node: np[int64, 12]}."""
    agg, state = {}, {}
    all_nodes = set(adj.keys()) | {c for kids in adj.values() for c in kids}

    for root in list(all_nodes):
        if state.get(root, 0) != 0:
            continue
        stack = [(root, 0)]
        while stack:
            node, phase = stack.pop()
            if phase == 0:
                if state.get(node, 0) in (1, 2):
                    continue
                state[node] = 1
                stack.append((node, 1))
                stack.extend((ch, 0) for ch in adj.get(node, []))
            else:
                subtotal = node_vec.get(node, np.zeros(len(MONTH_KEYS), dtype=np.int64)).copy()
                for ch in adj.get(node, []):
                    subtotal += agg.get(ch, np.zeros(len(MONTH_KEYS), dtype=np.int64))
                agg[node] = subtotal
                state[node] = 2
    return agg

# ── Main ────────────────────────────────────────────────────────────────────
def main():
    # 1) Feeder trace --------------------------------------------------------
    df = pd.read_csv(IN_FEDDER, low_memory=False)
    df["FEEDER_ID"] = df["FEEDER_ID"].astype(str).str.strip()


    if "DT_LOAD_CAL" not in df.columns:
        den = 11 * 0.95 * np.sqrt(3.0)
        df["DT_LOAD_CAL"] = pd.to_numeric(df.get("DT_LOAD", np.nan), errors="coerce") / den

    # 2) FRACTION per feeder -------------------------------------------------
    uniq = (df[["FEEDER_ID", "LOCATION", "DT_LOAD_CAL"]]
            .dropna(subset=["DT_LOAD_CAL"])
            .groupby(["FEEDER_ID", "LOCATION"], as_index=False)
            .agg(DT_LOAD_CAL=("DT_LOAD_CAL", "sum")))
    sum_by_fid = uniq.groupby("FEEDER_ID")["DT_LOAD_CAL"].sum().rename("SUM_DT_LOAD_CAL")
    uniq = uniq.merge(sum_by_fid, on="FEEDER_ID")
    uniq["FRACTION"] = uniq["DT_LOAD_CAL"] / uniq["SUM_DT_LOAD_CAL"].replace(0, np.nan)

    df = df.merge(
        uniq.rename(columns={"LOCATION": "DESTINATION_LOCATION"})
            [["FEEDER_ID", "DESTINATION_LOCATION", "SUM_DT_LOAD_CAL", "FRACTION"]],
        on=["FEEDER_ID", "DESTINATION_LOCATION"],
        how="left"
    )

    # 3) Feeder monthly averages -------------------------------------------
    matrix = pd.read_csv(IN_MATRIX, low_memory=False)
    feed_long = make_feed_long(matrix)             # long form
    feed_wide = (feed_long.pivot(index="FEEDER_ID", columns="MONTH",
                                 values="FEEDER_LOAD_AVG")
                           .reindex(columns=MONTH_KEYS, fill_value=0.0)
                           .reset_index())
    feed_wide.columns = ["FEEDER_ID"] + FEED_COLS
    df = df.merge(feed_wide, on="FEEDER_ID", how="left")

    # 4) Per-node integer monthly loads ------------------------------------
    out_nodes = (uniq[["FEEDER_ID", "LOCATION", "FRACTION"]]
                 .merge(feed_long, on="FEEDER_ID"))
    out_nodes["NODE_LOAD"] = out_nodes["FRACTION"].fillna(0) * out_nodes["FEEDER_LOAD_AVG"]

    node_wide = (out_nodes.pivot(index=["FEEDER_ID", "LOCATION"], columns="MONTH",
                                 values="NODE_LOAD")
                           .reindex(columns=MONTH_KEYS, fill_value=0)
                           .reset_index())
    node_wide[MONTH_KEYS] = np.ceil(node_wide[MONTH_KEYS].astype(float)).astype("Int64")

    node_wide_out = node_wide.rename(columns={"LOCATION": "DESTINATION_LOCATION"})
    node_wide_out.columns = ["FEEDER_ID", "DESTINATION_LOCATION"] + NODE_COLS
    df = df.merge(node_wide_out, on=["FEEDER_ID", "DESTINATION_LOCATION"], how="left")

    # 5) Bottom-up cumulative section loads --------------------------------
    sec_parts = []
    for fid, gdf in df.groupby("FEEDER_ID", sort=False):
        adj = build_graph(gdf[["SOURCE_LOCATION", "DESTINATION_LOCATION"]])
        sub_nodes = node_wide[node_wide["FEEDER_ID"] == fid]
        node_vec = {str(loc): sub_nodes.loc[sub_nodes["LOCATION"] == loc, MONTH_KEYS]
                                     .to_numpy(dtype="int64").reshape(-1, len(MONTH_KEYS))[0]
                    for loc in sub_nodes["LOCATION"].astype(str).unique()}
        for n in set(adj.keys()) | {c for kids in adj.values() for c in kids}:
            node_vec.setdefault(n, np.zeros(len(MONTH_KEYS), dtype=np.int64))

        agg = dfs_subtree_cum(adj, node_vec)
        rows = []
        for loc, vec in agg.items():
            cum = np.cumsum(vec, dtype=np.int64)
            rows.append({"FEEDER_ID": fid, "DESTINATION_LOCATION": loc,
                         **{f"sec_cum_{m}": int(cum[j]) for j, m in enumerate(MONTH_KEYS)}})
        sec_parts.append(pd.DataFrame(rows))

    sec_all = pd.concat(sec_parts, ignore_index=True)
    df = df.merge(sec_all, on=["FEEDER_ID", "DESTINATION_LOCATION"], how="left")

    # 6) Write --------------------------------------------------------------
    df.to_csv(OUT_FILE, index=False)
    print(f"Wrote -> {OUT_FILE}")

if __name__ == "__main__":
    main()


Wrote -> /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv


In [4]:
df= pd.read_csv("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")
col = df['FEEDER_ID'].unique()
print(len(col))

1099


RECURSIVE AND ITTERATIVE BOTH 
FUNCTION DFS

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
11 kV feeder ETL: node-level monthly loads + cumulative section flows.

Choose aggregation style with AGG_METHOD:
    "iter"  – explicit-stack DFS (depth-safe, quiet on cycles)
    "rec"   – recursive DFS   (simpler; self-loops skipped, other cycles → ValueError)
"""

from __future__ import annotations
from pathlib import Path
import re, sys
import numpy as np
import pandas as pd

# ── USER CHOICE ────────────────────────────────────────────────────────────
AGG_METHOD  = "re"      # "iter"  or  "rec"
DEPTH_LIMIT = 20_000      # only used if AGG_METHOD == "rec"

# ── Paths ───────────────────────────────────────────────────────────────────
BASE      = "/media/sagark24/New Volume/MERGE CDIS"
IN_FEDDER = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/11_WITHOUTDT_CONNECTED_RANKED.csv")
IN_MATRIX = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/monthly_SWNO_matrix_11KV.csv")
OUT_FILE  = Path(f"{BASE}/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv")

# ── Discover columns Month_01 … Month_12 ───────────────────────────────────
hdr = pd.read_csv(IN_MATRIX, nrows=0).columns
MONTH_KEYS = sorted([c for c in hdr if re.fullmatch(r"Month_\d{2}", c)],
                    key=lambda x: int(x.split("_")[1]))
if not MONTH_KEYS:
    raise RuntimeError("No Month_XX columns in matrix!")

FEED_COLS = [f"feeder_{m}"    for m in MONTH_KEYS]
NODE_COLS = [f"node_load_{m}" for m in MONTH_KEYS]
CUM_COLS  = [f"sec_cum_{m}"   for m in MONTH_KEYS]

# ── Helpers ────────────────────────────────────────────────────────────────
def make_feed_long(mat: pd.DataFrame) -> pd.DataFrame:
    """SWNO matrix → long FEEDER_ID | MONTH | FEEDER_LOAD_AVG (fills 0)."""
    need = {"SWNO", *MONTH_KEYS}
    miss = [c for c in need if c not in mat.columns]
    if miss:
        raise RuntimeError(f"matrix missing {miss}")

    m = mat.copy()
    m["FEEDER_ID"] = m["SWNO"].astype(str).str.strip()
    m[MONTH_KEYS]  = m[MONTH_KEYS].apply(pd.to_numeric, errors="coerce").fillna(0.0)

    long = m.melt(id_vars=["FEEDER_ID"], value_vars=MONTH_KEYS,
                  var_name="MONTH", value_name="FEEDER_LOAD_AVG")

    complete = pd.MultiIndex.from_product(
        [m["FEEDER_ID"].unique(), MONTH_KEYS],
        names=["FEEDER_ID","MONTH"]
    ).to_frame(index=False)

    return (complete.merge(long, on=["FEEDER_ID","MONTH"], how="left")
                  .fillna({"FEEDER_LOAD_AVG": 0.0}))

def build_adj(df_f: pd.DataFrame) -> dict[str, list[str]]:
    """parent -> [children] (dedup, leaf keys included)."""
    adj: dict[str,list[str]] = {}
    for s,d in zip(df_f["SOURCE_LOCATION"].astype(str),
                   df_f["DESTINATION_LOCATION"].astype(str)):
        adj.setdefault(s, []).append(d)
        adj.setdefault(d, [])
    for k in adj:
        adj[k] = list(dict.fromkeys(adj[k]))
    return adj

# ── Aggregation  explicit-stack DFS ───────────────────────────────────────
def agg_iter(adj: dict[str,list[str]], node_vec) -> dict[str,np.ndarray]:
    agg, state = {}, {}          # 0 unseen • 1 visiting • 2 done
    all_nodes  = set(adj.keys()) | {c for kids in adj.values() for c in kids}

    for start in all_nodes:
        if state.get(start,0):
            continue
        stack = [(start,0)]
        while stack:
            n, ph = stack.pop()
            if ph==0:
                if state.get(n,0):                       # visiting/done
                    continue
                state[n]=1
                stack.append((n,1))
                stack.extend((c,0) for c in adj[n] if c!=n)  # skip self-loop
            else:
                v = node_vec.get(n, np.zeros_like(next(iter(node_vec.values())))).copy()
                for c in adj[n]:
                    v += agg.get(c, np.zeros_like(v))
                agg[n]=v; state[n]=2
    return agg

# ── Aggregation  recursive DFS (self-loop safe) ───────────────────────────
def agg_rec(adj: dict[str,list[str]], node_vec,
            depth_limit: int = 20_000) -> dict[str,np.ndarray]:
    sys.setrecursionlimit(depth_limit)

    parents  = set(adj.keys())
    children = {c for kids in adj.values() for c in kids}
    roots    = parents - children or parents      # fallback

    done, visiting = {}, set()
    zeros_proto = np.zeros_like(next(iter(node_vec.values())))

    def dfs(v:str)->np.ndarray:
        if v in done:        return done[v]
        if v in visiting:    raise ValueError(f"cycle at {v}")
        visiting.add(v)
        s = node_vec.get(v, zeros_proto).copy()
        for c in adj[v]:
            if c!=v:
                s += dfs(c)
        visiting.remove(v)
        done[v] = s
        return s

    for r in roots: dfs(r)
    return done

# ── Pipeline ───────────────────────────────────────────────────────────────
def main():
    # 1) feeder trace -------------------------------------------------------
    df = pd.read_csv(IN_FEDDER, low_memory=False)
    df["FEEDER_ID"] = df["FEEDER_ID"].astype(str).str.strip()

    if "DT_LOAD_CAL" not in df.columns:
        den = 11 * 0.95 * np.sqrt(3.0)
        df["DT_LOAD_CAL"] = pd.to_numeric(df.get("DT_LOAD"), errors="coerce") / den

    # 2) FRACTION per feeder -------------------------------------------------
    uniq = (df[["FEEDER_ID","LOCATION","DT_LOAD_CAL"]]
              .dropna(subset=["DT_LOAD_CAL"])
              .groupby(["FEEDER_ID","LOCATION"], as_index=False)
              .agg(DT_LOAD_CAL=("DT_LOAD_CAL","sum")))
    total = uniq.groupby("FEEDER_ID")["DT_LOAD_CAL"].sum().rename("SUM_DT_LOAD_CAL")
    uniq = uniq.merge(total, on="FEEDER_ID")
    uniq["FRACTION"] = uniq["DT_LOAD_CAL"] / uniq["SUM_DT_LOAD_CAL"].replace(0,np.nan)

    df = df.merge(
        uniq.rename(columns={"LOCATION":"DESTINATION_LOCATION"})
            [["FEEDER_ID","DESTINATION_LOCATION","SUM_DT_LOAD_CAL","FRACTION"]],
        on=["FEEDER_ID","DESTINATION_LOCATION"], how="left"
    )

    # 3) feeder monthly averages -------------------------------------------
    feed_long = make_feed_long(pd.read_csv(IN_MATRIX, low_memory=False))
    feed_wide = (feed_long.pivot(index="FEEDER_ID", columns="MONTH",
                                 values="FEEDER_LOAD_AVG")
                         .reindex(columns=MONTH_KEYS, fill_value=0.0)
                         .reset_index())
    feed_wide.columns = ["FEEDER_ID"] + FEED_COLS
    df = df.merge(feed_wide, on="FEEDER_ID", how="left")

    # 4) per-node int monthly loads ----------------------------------------
    out_nodes = (uniq[["FEEDER_ID","LOCATION","FRACTION"]]
                 .merge(feed_long, on="FEEDER_ID"))
    out_nodes["NODE_LOAD"] = out_nodes["FRACTION"].fillna(0)*out_nodes["FEEDER_LOAD_AVG"]

    node_wide = (out_nodes.pivot(index=["FEEDER_ID","LOCATION"], columns="MONTH",
                                 values="NODE_LOAD")
                           .reindex(columns=MONTH_KEYS, fill_value=0)
                           .reset_index())
    node_wide[MONTH_KEYS] = np.ceil(node_wide[MONTH_KEYS].astype(float)).astype("Int64")

    node_wide_out = node_wide.rename(columns={"LOCATION":"DESTINATION_LOCATION"})
    node_wide_out.columns = ["FEEDER_ID","DESTINATION_LOCATION"] + NODE_COLS
    df = df.merge(node_wide_out, on=["FEEDER_ID","DESTINATION_LOCATION"], how="left")

    # 5) bottom-up cumulative ----------------------------------------------
    sec_parts=[]
    for fid,gdf in df.groupby("FEEDER_ID", sort=False):
        adj = build_adj(gdf[["SOURCE_LOCATION","DESTINATION_LOCATION"]])
        sub = node_wide[node_wide["FEEDER_ID"]==fid]
        node_vec = {str(l): sub.loc[sub["LOCATION"]==l, MONTH_KEYS]
                                   .to_numpy(dtype="int64").reshape(-1,len(MONTH_KEYS))[0]
                    for l in sub["LOCATION"].astype(str).unique()}
        for n in set(adj.keys())|{c for kids in adj.values() for c in kids}:
            node_vec.setdefault(n, np.zeros(len(MONTH_KEYS),dtype=np.int64))

        # ── pick aggregation strategy correctly ───────────────────────────
        if AGG_METHOD == "iter":
            agg = agg_iter(adj, node_vec)                         # ← two args
        else:
            agg = agg_rec(adj, node_vec, DEPTH_LIMIT)             # ← three args


        rows=[]
        for loc,vec in agg.items():
            cum=np.cumsum(vec, dtype=np.int64)
            rows.append({"FEEDER_ID":fid,"DESTINATION_LOCATION":loc,
                         **{f"sec_cum_{m}":int(cum[i]) for i,m in enumerate(MONTH_KEYS)}})
        sec_parts.append(pd.DataFrame(rows))

    df = df.merge(pd.concat(sec_parts, ignore_index=True),
                  on=["FEEDER_ID","DESTINATION_LOCATION"], how="left")

    # 6) write --------------------------------------------------------------
    df.to_csv(OUT_FILE, index=False)
    print(f" Wrote using AGG_METHOD='{AGG_METHOD}' -> {OUT_FILE}")

if __name__=="__main__":
    main()


 Wrote using AGG_METHOD='re' → /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/11_KV_FEEDER_ALL DATA/fedder_11KV_with_all_monthly_loads.csv
